In [86]:
import pandas as pd
from selenium import webdriver
import os
import time
import urllib2
from bs4 import BeautifulSoup

news=pd.read_csv('/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/data/kaggle/uci-news-aggregator.csv')

In [87]:
path_to_phatomJS='/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/flaskapp/phantomjs'

def getPageData(URL):
    browser = webdriver.PhantomJS(executable_path = path_to_phatomJS,service_log_path=os.path.devnull)
    browser.get(URL)
    pageContents=browser.page_source
    browser.close()
    browser.quit()
    return(pageContents)

def getTextData(URL):
    FullData=[]
    pageContents=getPageData(URL)
    if(len(pageContents) > 400):
        soup = BeautifulSoup(pageContents, "html.parser")
        ignoreList=['html','head','script','meta','link','style','body','header','img']
        ignoreWords=['CDATA','function']
        for x in soup.find_all():
            if(x.name not in ignoreList):
                if(len(x.text) > 200 and 'CDATA' not in x.text and 'function' not in x.text):
                    FullData.append(' '.join(x.text.strip().replace('\n',' ').split()))
        FullData=list(set(FullData))
    return(FullData)

In [ ]:
import time
FullData=pd.DataFrame(columns=['text','link'])
fileCounter=0
for link in news['URL'].values:
    try:
        data=pd.DataFrame(getTextData(link),columns=['text'])
        data['link']=link
        FullData=pd.concat([FullData,data])
        if(FullData.shape[0] > 100000):
            print("Writing to file {} with shape {}".format(fileCounter,FullData.shape[0]))
            FullData['text']=FullData['text'].map(lambda x : ''.join([y for y in x if ord(y) >= 32 and ord(y) <= 126]))
            FullData.to_csv('/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/data/kaggle/uciNews_{}.txt'.format(fileCounter),index=False)
            FullData=pd.DataFrame(columns=['text','link'])
            fileCounter=fileCounter + 1
            print("Written to file")
    except:
        print("There is some error. Will sleep for 30 seconds")
        time.sleep(30)
print("Writing to file {} with shape {}".format(fileCounter,FullData.shape[0]))        
FullData['text']=FullData['text'].map(lambda x : ''.join([y for y in x if ord(y) >= 32 and ord(y) < 126]))
FullData.to_csv('/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/data/kaggle/uciNews_{}.txt'.format(fileCounter),index=False)
fileCounter=fileCounter + 1